In [1]:
import torch
import numpy as np
import pandas as pd
from hyper import HyperNetwork
from rbergomi import rb_pricing
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
torch.__version__

'2.0.0'

In [3]:
torch.manual_seed(1943)

In [4]:
pricing_network = torch.nn.Sequential(
    torch.nn.Linear(3, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 1),
)

In [5]:
# compute the total number of parameters in the pricing network
n_params = sum([p.numel() for p in pricing_network.parameters()])
n_params

501

In [6]:
hyper_network = torch.nn.Sequential(
    torch.nn.Linear(4, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, n_params),
)

In [7]:
hyper_network.load_state_dict(torch.load('trained_hyper.pth'))

<All keys matched successfully>

In [8]:
hyper_network.eval()

Sequential(
  (0): Linear(in_features=4, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=501, bias=True)
)

In [9]:
model = HyperNetwork(hyper_network, pricing_network)

In [10]:
df = pd.read_json('./sp500_2017.json.bz2', compression='bz2', orient='index')

In [11]:
# df

In [12]:
one_day = df.query('date=="2017-12-29"')

In [13]:
options = one_day[['forward_price','tau','strike_price', 'is_call']].values

In [14]:
input_all = np.vstack([options[:,2]/options[:,0], options[:,1], options[:,3]]).T.astype(np.float32)

In [15]:
price_norm = one_day['option_price'].values 
price_norm = price_norm * np.exp(one_day['risk_free_rate'].values * one_day['tau'].values)
price_norm = price_norm / one_day['forward_price'].values
price_norm = price_norm.astype(np.float32)

In [16]:
mf_model_param_unconstrained = torch.randn(4, requires_grad=True)

In [17]:
optim = torch.optim.Adam([mf_model_param_unconstrained], lr=1e-3)

In [18]:
for I in range(1000):
    
    H = torch.sigmoid(mf_model_param_unconstrained[0])*0.5      # (0, 0.5)
    rho = torch.tanh(mf_model_param_unconstrained[1])           # (-1,1)
    eta = torch.sigmoid(mf_model_param_unconstrained[2])*5.0    # (0, 5]
    v0 = torch.sigmoid(mf_model_param_unconstrained[3])         # (0, 1]
    
    mf_model_param = torch.stack([H, rho, eta, v0]).view(1,-1)
    
    model_pred = model(mf_model_param, torch.from_numpy(input_all[None,:,:])).squeeze(0,2)
    loss = torch.abs(model_pred-torch.from_numpy(price_norm)).mean()

    optim.zero_grad()
    loss.backward()
    optim.step()
    
    if I % 100 == 0:
        print(loss.detach().cpu().numpy())

0.6235524
0.55807966
0.5054046
0.45851958
0.41802096
0.37603143
0.34515008
0.31881317
0.29620075
0.27432784


In [19]:
pred_prices = rb_pricing(input_all,
                         1.0, 
                         H.detach().cpu().numpy(), 
                         rho.detach().cpu().numpy(), 
                         eta.detach().cpu().numpy(),
                         v0.detach().cpu().numpy())

In [20]:
pred_prices_final = pred_prices['output'].flatten()
pred_prices_final = pred_prices_final / np.exp(one_day['risk_free_rate'].values * one_day['tau'].values)
pred_prices_final = pred_prices_final * one_day['forward_price'].values

In [21]:
# pred_prices_final

In [22]:
# plt.scatter(one_day['strike_price'], one_day['option_price'], marker='.')
# plt.scatter(one_day['strike_price'], pred_prices_final, marker='.')